In [ ]:
import pandas as pd

df = pd.read_json('./friends-1-227-Rachel-pair.json')

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

def train():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')

    # Tokenize the conversations and create input/output pairs
    input_ids = []
    output_ids = []

    inputs = df['question']
    outputs = df['answer']

    for i in range(df.shape[0] - 1):
        input_text = "<|startoftext|>" + inputs[i] + "<|endoftext|>"
        output_text = outputs[i]

        input_tokenized = tokenizer.encode(input_text,
                add_special_tokens=False)
        output_tokenized = tokenizer.encode(output_text,
                add_special_tokens=False)
        input_ids.append(input_tokenized)
        output_ids.append(output_tokenized)

    # Pad the input/output pairs to the same length
    max_length = max(len(ids) for ids in input_ids + output_ids)

    input_ids = \
        tf.keras.preprocessing.sequence.pad_sequences(input_ids,
            maxlen=max_length, padding='post')
    output_ids = \
        tf.keras.preprocessing.sequence.pad_sequences(output_ids,
            maxlen=max_length, padding='post')

    # Define the training parameters
    batch_size = 16
    epochs = 3
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

    # Compile the model
    model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

    # Train the model
    model.fit(input_ids, output_ids, batch_size=batch_size,
              epochs=epochs)

    # Save the trained model
    model.save_pretrained('chatbot_model')

In [ ]:
import numpy as np

def test():
  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  model = TFGPT2LMHeadModel.from_pretrained("chatbot_model")

  input_ids = []
  output_ids = []

  inputs = df['question']
  outputs = df['answer']

  inputs = df['question']
  outputs = df['answer']

  for i in range(df.shape[0] - 1):
      input_text = inputs[i]
      output_text = outputs[i]

      input_tokenized = tokenizer.encode(input_text,
              add_special_tokens=False)
      output_tokenized = tokenizer.encode(output_text,
              add_special_tokens=False)
      input_ids.append(input_tokenized)
      output_ids.append(output_tokenized)

  # Pad the input/output pairs to the same length
  max_length = max(len(ids) for ids in input_ids + output_ids)

  while True:
    input_text = input("User: ")
    input_text = "<|startoftext|>" + input_text + "<|endoftext|>"
    input_tokenized = tokenizer.encode(input_text, add_special_tokens=False)
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_tokenized], maxlen=max_length, padding="post")

    attention_mask = np.where(input_ids != 0, 1, 0)  # Dikkat maskesi oluştur
    output_ids = model.generate(input_ids, max_length=max_length + 1, attention_mask=attention_mask, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Burada gereksiz tekrarları ve anlamsız tokenları filtreleyebilirsiniz

    print("Bot:", output_text)

In [ ]:
train()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
285/285 [==============================] - 74s 139ms/step - loss: 1.4485
Epoch 2/3
285/285 [==============================] - 40s 140ms/step - loss: 1.3675
Epoch 3/3
285/285 [==============================] - 40s 140ms/step - loss: 1.3528


In [ ]:
test()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at chatbot_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


User: hi


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: <|startoftext|>hi!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!.
User: hello


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: <|startoftext|>hello!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!.


KeyboardInterrupt: Interrupted by user